# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bom jardim,-7.7958,-35.5872,25.31,58,97,7.12,BR,1722535602
1,1,karpinsk,59.7654,60.0085,14.14,96,97,1.38,RU,1722535604
2,2,olonkinbyen,70.9221,-8.7187,5.08,99,100,5.35,SJ,1722535605
3,3,sisimiut,66.9395,-53.6735,11.82,52,2,3.26,GL,1722535606
4,4,nuuk,64.1835,-51.7216,15.34,41,0,8.23,GL,1722535572


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x = "Lng",
    y = "Lat",
    size = "Humidity",
    geo = True,
    tiles = "OSM",
    color = "City",
    title = "City Map based on Humidity",
    xlabel = "Longitude",
    ylabel = "Latitude"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [22]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] >= 21) & (city_data_df["Max Temp"] < 27) &  
    (city_data_df["Humidity"] < 50) &
    (city_data_df["Cloudiness"] < 45)
]
# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()
;
# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
85,85,fort mcmurray,56.7268,-111.3810,22.85,43,10,2.24,CA,1722535699
124,124,maloye ganyushkino,46.6000,49.2667,26.28,30,36,3.86,KZ,1722535749
137,137,la ronge,55.1001,-105.2842,21.00,46,2,6.87,CA,1722535763
158,158,calama,-22.4667,-68.9333,23.27,9,0,10.80,CL,1722535659
192,192,kalnibolotskaya,46.0058,40.4568,23.15,34,29,3.84,RU,1722535834
277,277,urcos,-13.6861,-71.6228,21.94,32,33,2.77,PE,1722535878
469,469,albertina,-22.2008,-46.6158,26.76,23,0,3.16,BR,1722536176
487,487,mwene-ditu,-7.0000,23.4500,25.88,18,2,0.44,CD,1722536198
502,502,concepcion del oro,24.6333,-101.4167,21.93,30,0,3.01,MX,1722536215
564,564,red deer,52.2668,-113.8020,23.49,40,0,7.72,CA,1722536292


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
85,fort mcmurray,CA,56.7268,-111.3810,43,
124,maloye ganyushkino,KZ,46.6000,49.2667,30,
137,la ronge,CA,55.1001,-105.2842,46,
158,calama,CL,-22.4667,-68.9333,9,
192,kalnibolotskaya,RU,46.0058,40.4568,34,
277,urcos,PE,-13.6861,-71.6228,32,
469,albertina,BR,-22.2008,-46.6158,23,
487,mwene-ditu,CD,-7.0000,23.4500,18,
502,concepcion del oro,MX,24.6333,-101.4167,30,
564,red deer,CA,52.2668,-113.8020,40,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [24]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data


Starting hotel search
fort mcmurray - nearest hotel: Nomad Hotel and Suites
maloye ganyushkino - nearest hotel: Конил Сазы
la ronge - nearest hotel: La Ronge Hotel and Suites
calama - nearest hotel: Apart Hotel Geotel Calama
kalnibolotskaya - nearest hotel: No hotel found
urcos - nearest hotel: Hospedaje Rafael
albertina - nearest hotel: Hotel Parque das Primaveras
mwene-ditu - nearest hotel: Hôtel KAMAS
concepcion del oro - nearest hotel: No hotel found
red deer - nearest hotel: Baymont By Wyndham Red Deer


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [25]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    x = "Lng",
    y = "Lat",
    size = "Humidity",
    geo = True,
    color = "City",
    tiles = "OSM",
    title = "Hotel Map based on Ideal Weather",
    xlabel = "Longitude",
    ylabel = "Latitude",
    hover_cols = ["Hotel Name", "Country"]
)
# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)